# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `[url_slug].md` and the permalink will be `https://[yourdomain]/talks/[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

title	type	authors	url_slug	venue	date	location	talk_url	description
ADHD individuals and media multitaskers exhibit tonic resource allocation differences When processing educational multimedia messages	Conference Paper	Fisher, J.T. & Keene, J.R.	ADHD_NCA18	The 104th Annual Meeting of the National Communication Association	2018-11	Salt Lake City UT, USA		
The moral foundations dictionary for news (MFD-N): A crowd-sourced moral foundations dictionary for the automated analysis of news corpora. 	Conference Paper	Hopp, F.R., Cornell, D., Fisher, J. T., Huskey, R., & Weber, R. 	MFD_NCA18	The 104th Annual Meeting of the National Communication Association	2018-11	Salt Lake City UT, USA		
" ""But how are they talked about?”: A novel measure of entity framing in online news"	Conference Paper	Fisher, J.T., Cornell, D., Hopp, F.R., Mangus, M., & Weber, R. 	Framing_ICA18	The 68th Annual Meeting of the International Communication Association	2018-05	Prague, CZ		
"""Look harder!"" ADHD, media multi

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,authors,url_slug,venue,date,location,talk_url,description
0,ADHD individuals and media multitaskers exhibi...,Conference Paper,"Fisher, J.T. & Keene, J.R.",ADHD_NCA18,The 104th Annual Meeting of the National Commu...,2018-11,"Salt Lake City UT, USA",NaN,NaN
1,The moral foundations dictionary for news (MFD...,Conference Paper,"Hopp, F.R., Cornell, D., Fisher, J. T., Huskey...",MFD_NCA18,The 104th Annual Meeting of the National Commu...,2018-11,"Salt Lake City UT, USA",NaN,NaN
2,"""But how are they talked about?”: A novel mea...",Conference Paper,"Fisher, J.T., Cornell, D., Hopp, F.R., Mangus,...",Framing_ICA18,The 68th Annual Meeting of the International C...,2018-05,"Prague, CZ",NaN,NaN
3,"""Look harder!"" ADHD, media multitasking, and t...",Conference Paper,"Fisher, J.T. Hopp, F.R., & Weber, R.",CogPerc_ICA18,The 68th Annual Meeting of the International C...,2018-05,"Prague, CZ",NaN,NaN
4,Organized complexity in communication science:...,Conference Paper,"Keene, J.R., Adams, R., Najera, C., Fisher. J....",Phys_ICA18,The 68th Annual Meeting of the International C...,2018-05,"Prague, CZ",NaN,NaN
5,Combining computational content analysis with ...,Conference Paper,"Hopp, F.R., Fisher, J.T., Mangus, J. M., Weber...",Sharing_ICA18,The 68th Annual Meeting of the International C...,2018-05,"Prague, CZ",NaN,NaN
6,"Mining the Global Database of Events, Language...",Conference Paper,"Hopp, F.R., Fisher, J.T., Mangus, J. M., Huske...",GDELT_ICA18,The 68th Annual Meeting of the International C...,2018-05,"Prague, CZ",NaN,NaN
7,Examining online news reporting using the Glob...,Conference Paper,"Hopp, F.R., Fisher, J.T., Mangus, J. M., Huske...",GDELT_NCA17,The 103rd Annual Meeting of the National Commu...,2017-11,"Dallas TX, USA",NaN,NaN
8,The offramp problem: Treating media psychology...,Conference Paper,"Fisher, J.T., Huskey, R., Keene, J. & Weber, R.",Offramp_ICA17,The 67th Annual Meeting of the International C...,2017-05,"San Diego CA, USA",NaN,NaN
9,Examining cognitive load and recall for educat...,Conference Paper,"Fisher, J.T., & Keene, J.R.",ADHD_ICA17,The 67th Annual Meeting of the International C...,2017-05,"San Diego CA, USA",NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = item.url_slug + ".md"
    html_filename = item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.authors)) > 3:
        md += 'authors: "' + item.authors + '"\n'
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2012-03-01-talk-1.md	  ADHD_ICA17.md     Emotion_SPR16.md  MFD_NCA18.md
2013-03-01-tutorial-1.md  ADHD_NCA18.md     Framing_ICA18.md  Offramp_ICA17.md
2014-02-01-talk-2.md	  CogPerc_ICA18.md  GDELT_ICA18.md    Phys_ICA18.md
2014-03-01-talk-3.md	  CTA_NCA17.md	    GDELT_NCA17.md    Sharing_ICA18.md


In [8]:
!cat ../_talks/GDELT_NCA17.md

---
title: "Examining online news reporting using the Global Database of Events, Language, and Tone: Challenges, opportunities, and future directions"
collection: talks
type: "Conference Paper"
permalink: /talks/GDELT_NCA17
authors: "Hopp, F.R., Fisher, J.T., Mangus, J. M., Huskey, R., Swanson, R., Gordon, A., Khooshabeh, P., Weber, R. "
venue: "The 103rd Annual Meeting of the National Communication Association"
date: 2017-11
location: "Dallas TX, USA"
---
